In [11]:
%matplotlib inline
import numpy as np
from scipy.optimize import minimize, LinearConstraint
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

## OPTIMIZATION OF SELLING N STOCKS TO M BUYERS

In [12]:
a = np.arange(10)
np.where(a < 5)

(array([0, 1, 2, 3, 4], dtype=int64),)

In [13]:
n_buyers = 5
n_shares = 15

np.random.seed(10)
prices = np.random.random(n_buyers)
money_available = np.random.randint(1, 4, n_buyers)
n_shares_per_buyer = money_available / prices
print(prices, money_available, n_shares_per_buyer, sep="\n")

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701]
[1 2 2 3 1]
[ 1.29647768 96.37648808  3.15632537  4.00638948  2.00598984]


In [14]:
constraint = LinearConstraint(np.ones(n_buyers), lb=n_shares, ub=n_shares)
bounds = [(0, n) for n in n_shares_per_buyer]

In [15]:
def objective_function(x, prices):
    return -x.dot(prices)

Niter = 1
def callbackF(X):
    global Niter
    print(Niter,X)
    Niter += 1

In [16]:

class Simulator:
    def __init__(self, function):
        self.f = function # actual objective function
        self.num_calls = 0 # how many times f has been called
        self.callback_count = 1 # number of times callback has been called, also measures iteration count
        self.list_calls_inp = [] # input of all calls
        self.list_calls_res = [] # result of all calls
        self.decreasing_list_calls_inp = [] # input of calls that resulted in decrease
        self.decreasing_list_calls_res = [] # result of calls that resulted in decrease
        self.list_callback_inp = [] # only appends inputs on callback, as such they correspond to the iterations
        self.list_callback_res = [] # only appends results on callback, as such they correspond to the iterations

    def simulate(self, x, *args):
        """Executes the actual simulation and returns the result, while
        updating the lists too. Pass to optimizer without arguments or
        parentheses."""
        result = self.f(x, *args) # the actual evaluation of the function
        if not self.num_calls: # first call is stored in all lists
            self.decreasing_list_calls_inp.append(x)
            self.decreasing_list_calls_res.append(result)
            self.list_callback_inp.append(x)
            self.list_callback_res.append(result)
        elif result < self.decreasing_list_calls_res[-1]:
            self.decreasing_list_calls_inp.append(x)
            self.decreasing_list_calls_res.append(result)
        self.list_calls_inp.append(x)
        self.list_calls_res.append(result)
        self.num_calls += 1
        return result

    def callback(self, xk, *_):
        """Callback function that can be used by optimizers of scipy.optimize.
        The third argument "*_" makes sure that it still works when the
        optimizer calls the callback function with more than one argument. Pass
        to optimizer without arguments or parentheses."""
        s1 = ""
        xk = np.atleast_1d(xk)
        # search backwards in input list for input corresponding to xk
        for i, x in reversed(list(enumerate(self.list_calls_inp))):
            x = np.atleast_1d(x)
            if np.allclose(x, xk):
                break

        for comp in xk:
            s1 += f"{comp:10.5e}\t"
        s1 += f"{self.list_calls_res[i]:10.5e}"

        self.list_callback_inp.append(xk)
        self.list_callback_res.append(self.list_calls_res[i])

        if not self.callback_count:
            s0 = ""
            for j, _ in enumerate(xk):
                tmp = f"Comp-{j+1}"
                s0 += f"{tmp:10s}\t"
            s0 += "Objective"
            print(s0)
            print(s1)

## 2D PCA OF OPTIMIZATION PATH + COST LANDSCAPE

In [20]:
def get_ranges(pca,inv_trans_X,trans_X,steps):
    ax_ranges = {i:None for i in range(len(pca.components_))}
    xmin = min(trans_X[:,0])
    xmax = max(trans_X[:,0])
    xrange = np.linspace(xmin,xmax,steps)
    ymin = min(trans_X[:,1])
    ymax = max(trans_X[:,1])
    yrange = np.linspace(ymin,ymax,steps)

    xrange = np.vstack((xrange,np.zeros(steps)+ymin))
    yrange = np.vstack((np.zeros(steps)+xmin,yrange))
    return [xrange,yrange]
    
#     for i,vector in enumerate(pca.components_):
#         # get max distance for scalar projection 
#         projs = [(X-pca.mean_) for X in inv_trans_X]
#         projs2 = [(np.dot(p,vector)/np.dot(vector,vector))*vector for p in projs]
#         projs3 = [np.linalg.norm(projs2[i]-p) for i,p in enumerate(projs)]
#         if not ax_ranges[i]:
#             ax_ranges[i] = max(projs3)
#         else:
#             print("Vector twice?")
        
#     ax_data = []
#     for i,vector in enumerate(pca.components_):  
#         # vector projection (from mean to orthogonally beneat point in PCA line)
#         ax_path = np.linspace(pca.mean_-(vector*ax_ranges[(i+1)%2]),pca.mean_+(vector*ax_ranges[(i+1)%2]),steps)
#         ax_data.append(ax_path)
#     return ax_data
    
# get 2D pca representation
def pca_data(data,steps,d=2):
    pca = PCA(n_components=d)
    pca.fit(data)
    trans_X = pca.transform(data)
    inv_trans_X = pca.inverse_transform(trans_X)
    ax_ranges = get_ranges(pca,inv_trans_X,trans_X,steps)
    data_2D = []
    for i in ax_ranges[0]:
        for r in ax_ranges[1]:
            curv = i+(r)
            data_2D.append(curv)
    return inv_trans_X,trans_X,data_2D

def plot_pca2D(trans_X,inv_trans_X,trans_data_2D,data_2D,steps):
    idx = [0,steps-1,steps*(steps-1)]
    colors = ['red','yellow','black']

    plt.plot(trans_X[:,0],trans_X[:,1])
#     plt.scatter(trans_data_2D[:,0],trans_data_2D[:,1])
#     for i,x in enumerate(colors):
#         ix = idx[i]
#         print(data_2D[ix])
#         plt.scatter(trans_data_2D[ix][0],trans_data_2D[ix][1],color=x)
#     plt.show()

In [21]:
sim = Simulator(objective_function)
x00=10 * np.random.random(n_buyers)
res = minimize(
    sim.simulate,
    x0=x00,
    args=(prices,),
    constraints=constraint,
    bounds=bounds,
    callback=sim.callback
)

a = sim.decreasing_list_calls_inp
b = sim.list_calls_inp
c = [list(x00)]+np.array(sim.list_callback_inp).tolist()
cur = c

In [22]:
range_steps = 10
inv_trans_X,trans_X,grid_nD,grid_2D,axes_2D = pca_data(cur,range_steps)

idx = [0,range_steps-1,range_steps*(range_steps-1)]
ax_x = axes_2D[0][:,0]
ax_y = axes_2D[1][:,1]
X, Y = np.meshgrid(ax_x,ax_y)
Z = []
for i in range(len(ax_x)):
    Z.append([])
    for j in range(len(ax_y)):
        indx = (range_steps*i)+j
        Z[-1].append(objective_function(grid_nD[indx],prices))
Z = np.array(Z)

plt.plot(trans_X[:,0],trans_X[:,1])
plt.scatter(trans_X[0][0],trans_X[0][1],color='red')
plt.scatter(trans_X[-1][0],trans_X[-1][1],color='green')
plt.contour(X, Y, Z, cmap='Spectral')
plt.colorbar()
plt.show()

ValueError: not enough values to unpack (expected 5, got 3)